In [13]:
import requests
from bs4 import BeautifulSoup
import pathlib
from datetime import datetime
import os
import time
import sys
sys.path.append('.')

from utils import *
from compress_utils import compressed_cpickle, decompressed_cpickle

In [3]:
# %%writefile ./compress_utils.py
# # data storage
# import _pickle as cpkl
# import bz2

# # Pickle a file and then compress it into a file with extension 
# """
# usage: 
#     compressed_pickle('path/filename', data) 
# """

# def compressed_cpickle(path_title, data):
#     with bz2.BZ2File(path_title+'.pbz2', "w") as f: 
#         cpkl.dump(data, f)

        
# # Load any compressed pickle file
# """
# usage: 
#     data = decompressed_cpickle('example_cp.pbz2') 
# """
# def decompressed_cpickle(file):
#     data = bz2.BZ2File(file, 'rb')
#     data = cpkl.load(data)
#     return data

In [4]:
# valid_website = ['openaccess.thecvf.com', 'proceedings.neurips.cc', 'link.springer.com', 'dl.acm.org',
#          'arxiv.org', 'www.pnas.org', 'www.usenix.org', 'www.sciencedirect.com', 'www.nature.com',
#          'ieeexplore.ieee.org', 'proceedings.mlr.press']

# # website = 'openaccess.thecvf.com'
# text_rule = lambda text:text.replace('.','_')
# extract_func_dic = {}
# for website in valid_website:
#     name_ = 'abs_from_'+text_rule(website)
#     extract_func_dic[name_] = eval(name_)
# extract_func_dic

In [5]:
def get_topic(soup, href=True):
    
    h3_soup = soup.find_all('h3', class_="gs_rt")
    gs_rs_soup = soup.find_all('div', class_="gs_rs")
    gs_a_group = soup.find_all('div', class_='gs_a')
    result = {'topic':[], "year":[]}
    
    valid_website = ['openaccess.thecvf.com', 'proceedings.neurips.cc', 'link.springer.com', 'dl.acm.org',
         'arxiv.org', 'www.pnas.org', 'www.usenix.org', 'www.sciencedirect.com', 'www.nature.com',
         'ieeexplore.ieee.org', 'proceedings.mlr.press']
    except_lis = ['ieeexplore.ieee.org']
    valid_website = [i for i in valid_website if not i in except_lis]
    
    text_rule = lambda text:text.replace('.','_')
    extract_func_dic = {}
    for website in valid_website:
        name_ = 'abs_from_'+text_rule(website)
        extract_func_dic[website] = eval(name_)
    

    if href:
        result['href'] = []
        
                
    # get topic and abstract and href
    for topic, abstract, attr in zip(h3_soup, gs_rs_soup, gs_a_group):
        # get hef:
        if href:
            a = str(topic).split(' ')
            href_ = [e[6:-1] for e in a if e.__contains__('href')][0]
    #             print(href_)
            result['href'].append(href_)
        
        # get year
        for i in [str(year) for year in range(2015,2022)]:
            if attr.get_text().__contains__(i):
                result['year'].append(i)
#                 print(i)
        
        # get topic
        topic_text = topic.get_text()
        b = '[HTML][HTML] '
        topic_text = topic_text.replace(b, '')
        
        

        
        # get abstract

        text_rule = lambda text:text.replace('.','_')
        
        for i in valid_website:
            if href_.__contains__(i):
                website = href_.split('/')[2]
                abstract = extract_func_dic[website](href_)
            else:
                pass

        result['topic'].append((topic_text, 123))
    
    return result

In [6]:
# def abs_from_ieeexplore_ieee_org(url): 
#     website = 'ieeexplore.ieee.org'
#     if url.__contains__(website):
#         pass
#     else:
#         raise KeyError(f'This function is only for web {website}')
#     r = requests.get(url)
#     abstract = r.text.split('meta property="og:description"')#[1].split('content="')[1].split('"')[0]
#     return abstract

In [7]:
# url = 'https://ieeexplore.ieee.org/abstract/document/9084352'
# # r
# # soup = BeautifulSoup(r.text, 'html.parser')
# # res = get_topic(soup)
# a = abs_from_ieeexplore_ieee_org(url)

In [23]:
google_url = "https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5"
end = 980
dic = {}
for i in range(end//10+1)[:1]:
    a = google_url + f'&start={i*10}'
    print(a)
    r = requests.get(a)
    
    soup = BeautifulSoup(r.text, 'html.parser')
    res = get_topic(soup)
    print(res)
#     lis.append()
    dic['page_'+str(i*10)] = res
    time.sleep(5)
    

https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5&start=0
{'topic': [('Ditto: Fair and robust federated learning through personalization', 123), ('Federated learning for internet of things: Recent advances, taxonomy, and open challenges', 123), ('Federated learning for predicting clinical outcomes in patients with COVID-19', 123), ('A survey on security and privacy of federated learning', 123), ('Oort: Efficient federated learning via guided participant selection', 123), ('A survey on federated learning', 123), ('Communication-efficient federated learning', 123), ('System optimization of federated learning networks with a constrained latency', 123), ('Federated learning based on dynamic regularization', 123), ('Multi-center federated learning', 123)], 'year': ['2021', '2021', '2021', '2021', '2021', '2021', '2021', '2021', '2021', '2021'], 'href': ['https://proceedings.mlr.press/v139/li21h.html', 'https://ieeexplore.ieee.org/abstract/document/94

In [24]:
from datetime import datetime
import pathlib

path = './Data'
pathlib.Path(f'{path}').mkdir(parents=True, exist_ok=True)  

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d%m-%H%M")
# print(dt_string)
compressed_cpickle(f'./Data/scholar_analysis_FL_{dt_string}', dic)


In [25]:
import os
import time

pbz_paths = ['/'.join([path,i]) for i in os.listdir(path) if i.__contains__('.pbz2')]
print(pbz_paths)
newest_file_path = max(pbz_paths, key=os.path.getctime)
newest_file_path

['./Data/scholar_analysis_FL_1401-1107.pbz2', './Data/scholar_analysis_FL_1401-1109.pbz2']


'./Data/scholar_analysis_FL_1401-1109.pbz2'

In [26]:
decompressed_cpickle(newest_file_path)

{'page_0': {'topic': [('Ditto: Fair and robust federated learning through personalization',
    123),
   ('Federated learning for internet of things: Recent advances, taxonomy, and open challenges',
    123),
   ('Federated learning for predicting clinical outcomes in patients with COVID-19',
    123),
   ('A survey on security and privacy of federated learning', 123),
   ('Oort: Efficient federated learning via guided participant selection',
    123),
   ('A survey on federated learning', 123),
   ('Communication-efficient federated learning', 123),
   ('System optimization of federated learning networks with a constrained latency',
    123),
   ('Federated learning based on dynamic regularization', 123),
   ('Multi-center federated learning', 123)],
  'year': ['2021',
   '2021',
   '2021',
   '2021',
   '2021',
   '2021',
   '2021',
   '2021',
   '2021',
   '2021'],
  'href': ['https://proceedings.mlr.press/v139/li21h.html',
   'https://ieeexplore.ieee.org/abstract/document/9460016/'

In [433]:
# url = 'https://scholar.google.com.tw/scholar?as_ylo=2021&q=federated+learning&hl=zh-TW&as_sdt=0,5&start=0'

# # headers = {'UserAgent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
# r = requests.get(url)
# r.text
# # soup = BeautifulSoup(r.text, 'html.parser')
# # soup

In [405]:
# r = requests.get(url)

def abs_from_www_sciencedirect_com(url): 
    
    website = 'www.sciencedirect.com'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')
    
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    r = requests.get(url, headers=headers)
    r.text
    soup = BeautifulSoup(r.text, 'html.parser')

    res = soup.find_all('div', class_ = 'abstract author')[0]
    

    return res.get_text()
abs_from_www_sciencedirect_com(url)

'AbstractFederated learning (FL) is a new breed of Artificial Intelligence (AI) that builds upon decentralized data and training that brings learning to the edge or directly on-device. FL is a new research area often referred to as a new dawn in AI, is in its infancy, and has not yet gained much trust in the community, mainly because of its (unknown) security and privacy implications. To advance the state of the research in this area and to realize extensive utilization of the FL approach and its mass adoption, its security and privacy concerns must be first identified, evaluated, and documented. FL is preferred in use-cases where security and privacy are the key concerns and having a clear view and understanding of risk factors enable an implementer/adopter of FL to successfully build a secure environment and gives researchers a clear vision on possible research areas. This paper aims to provide a comprehensive study concerning FL’s security and privacy aspects that can help bridge th

In [461]:
# r = requests.get(url)

def abs_from_academic_oup_com(url): 
    
    website = 'academic.oup.com'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')
        
        
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    res = soup.find_all('section', class_ = 'abstract')
    # res.find('div', class_='sec')
    abstract = ''
    for sec in res:
    #     sec.find('div', class_='sec')
        for sub_sec in sec:
            a = sub_sec.contents[0].get_text()
            b = sub_sec.contents[1].get_text()
            abstract += a+' '
            abstract += b
    return abstract
# abs_from_academic_oup_com(url)

In [460]:
def abs_from_openaccess_thecvf_com(url): 

    website = 'openaccess.thecvf.com'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    res = soup.find_all('div', id='abstract')
    abstract = ""
    for i in res:
#         print(i)
        abstract+= i.get_text()
    return abstract.replace('\n', '').strip() #.split('Abstract: ')[1]

# abs_from_openaccess_thecvf_com(url)

In [459]:
def abs_from_proceedings_neurips_cc(url): 

    website = 'proceedings.neurips.cc'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    res= soup.find_all('p')

    max_length = max([len(i.get_text()) for i in res])
    for i in res:
        if len(i.get_text()) == max_length:
#             print(i.get_text())  
            return i.get_text()

# abs_from_proceedings_neurips_cc(url)

In [458]:
def abs_from_link_springer_com(url): 

    website = 'link.springer.com'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    res = soup.find_all('div',  class_='c-article-section__content', id='Abs1-content')
    abstract = ""
    for i in res:
#         print(i)
        abstract+= i.get_text()
    return abstract #.split('Abstract: ')[1].replace('\n', '').strip()

# abs_from_link_springer_com(url)

In [457]:
def abs_from_dl_acm_org(url): 

    website = 'dl.acm.org'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    res = soup.find_all('div',  class_="abstractSection abstractInFull")
    abstract = ""
    for i in res:
#         print(i)
        abstract+= i.get_text()
    return abstract #.split('Abstract: ')[1].replace('\n', '').strip()

# abs_from_dl_acm_org(url)

In [456]:
def abs_from_arxiv_org(url): 

    website = 'arxiv.org'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    res = soup.find_all('blockquote',  class_="abstract mathjax")
    abstract = ""
    for i in res:
#         print(i)
        abstract+= i.get_text()
    return abstract.split('Abstract: ')[1].replace('\n', '').strip()

# abs_from_arxiv_org(url)

In [455]:
def abs_from_www_pnas_org(url): 

    website = 'www.pnas.org'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    res = soup.find_all('div',  class_="section abstract")
    abstract = ""
    for i in res:
    #         print(i.get_text())
        abstract+= i.get_text()
    return abstract

# abs_from_www_pnas_org(url)

In [454]:
def abs_from_www_usenix_org(url): 
    website = 'www.usenix.org'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')
        
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    res = soup.find_all('div',  class_="field field-name-field-paper-description field-type-text-long field-label-above")
    abstract = ""
    for i in res:
#         print(i.get_text())
        abstract+= i.get_text()
    return abstract
#     soup_ = BeautifulSoup(i, 'html.parser')

# abs_from_www_usenix_org(url)

In [453]:
# def abs_from_ieeexplore_ieee_org(url): 
# print(url)
def abs_from_www_nature_com(url): 
    website = 'www.nature.com'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    res = soup.find_all('div',  class_="c-article-section__content")
    return [ i.get_text().strip() for i in res][0]

# abs_from_www_nature_com(url)

In [448]:
# if url.__contains__(website):
#     print(url)
#     pass
def abs_from_ieeexplore_ieee_org(url): 
    website = 'ieeexplore.ieee.org'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')
    r = requests.get(url)
    abstract = r.text.split('meta property="og:description"')[1].split('content="')[1].split('"')[0]
    return abstract
# abs_from_ieeexplore_ieee_org(url)

In [449]:
def abs_from_proceedings_mlr_press(url):
    website = 'proceedings.mlr.press'
    if url.__contains__(website):
        pass
    else:
        raise KeyError(f'This function is only for web {website}')
    r = requests.get(url)

    soup = BeautifulSoup(r.text, 'html.parser')
    res = soup.find_all('div',  class_="abstract")
    return [ i.get_text().strip() for i in res][0]


# abs_from_proceedings_mlr_press(url)

In [447]:
# def get_des(dic_gs_rs):
#     des = []
#     for i in dic_gs_rs:
#         a = i.get_text()
#         des.append(a)
#     return des
# if __name__ == '__main__':
#     res = get_des(soup.find_all('div',  class_="gs_rs"))
#     print(res)